In [ ]:
import cv2
from PIL import Image, ImageDraw

In [ ]:
ui = GET_YOUR_MODEL()

In [ ]:
orginal_image = cv2.imread(r"img.jpg")
# img = ui.scan_image(orginal_image.copy())
img = Image.fromarray(orginal_image)
image_detected, texts, boxes,labels,scores = ui.predict_single(img, ocr=True)

In [ ]:
img

In [ ]:
image_detected

<!-- ### Steps of the grid algorithm -->

<!-- * Sort bounding boxes top to bottom, using ymin value
* Starting from the top, calculate the height intersection value of the current bounding box's height(ymax-ymin) and the next bounding box's height
* If the height intersection value is above the threshold value, draw a line between the current box's ymax and next box's ymin and keep row coordinates in a list. A higher intersection means they are in the same row.
* Iterate through boxes and determine which row they belong to by comparing each object's midpoint(height/2)
* Sort bounding boxes left to right, using xmin value
* Iterate through the bounding boxes of each row. Starting from the left, calculate the width intersection value of the current bounding box's width(xmax-xmin) and the next bounding box's width
* If the width intersection value is above the threshold value, draw a line between the current box's xmax and the next box's xmin and keep row coordinates in a list. A higher intersection means they are in the same column.
* Iterate through boxes and determine which column they belong to by comparing each object's midpoint(width/2) -->

<!-- ### Steps of the grouping algorithm -->
<!-- * Create a matrix of max(column index) x max(row index) representing the grid of the document.
* Iterate through cells of the matrix. If there is an object at that location in the grid assign its label code else assign -1
* For a given list of label codes to search, find groups by exploring adjacent elements with the same value recursively -->

In [ ]:
objects = [
    {"bbox":boxes[i],
     "label": model.get_reverse_label_dict()[labels[i]], 
     "score":scores[i],
     "text": texts[i] if texts and i in texts.keys() else None,
     "id":i,
     "alignment":None,
     "group_id": None,
     "pos":[]
     } for i in range(len(boxes))
]

In [ ]:
def draw_rows(draw, objects, row_intersection_limit):

    objects.sort(key=lambda obj: obj["bbox"][1])

    rows = []

    for i in range(len(objects)-1):
        
        # boxes = [obj["bbox"] for obj in objects]
        
        if (objects[i+1]["bbox"][1] > objects[i]["bbox"][3] or objects[i]["bbox"][1] > objects[i+1]["bbox"][3]):
            intersection = 0,0
        else:
            intersection = max(objects[i]["bbox"][1],objects[i+1]["bbox"][1]), min(objects[i]["bbox"][3],objects[i+1]["bbox"][3])

        x_height = objects[i]["bbox"][3] - objects[i]["bbox"][1]
        y_height = objects[i+1]["bbox"][3] - objects[i+1]["bbox"][1]
        intersection_height = intersection[1] - intersection[0]

        
        if not (intersection_height/x_height and intersection_height/y_height) > row_intersection_limit:
            
            row = (0,(objects[i]["bbox"][3]  + objects[i+1]["bbox"][1])/2, img.size[0],(objects[i]["bbox"][3]  + objects[i+1]["bbox"][1])/2)
            # draw.line(row, fill="#FF5C5C", width=10) 
            rows.append(row[1])

    return rows

In [ ]:
def draw_columns(draw, rows, objects, col_intersection_limit):

    objects.sort(key=lambda obj: obj["bbox"][0])

    cols_by_rows = {i: [] for i in range(len(rows)+ 1)}

    for j in range(len(rows)+1):

        row_objects = [objects[i] for i in range(len(objects)) if objects[i]["row"] == j] 


        for i in range(len(row_objects)-1):

            if (row_objects[i+1]["bbox"][0] > row_objects[i]["bbox"][2] or row_objects[i]["bbox"][0] > row_objects[i+1]["bbox"][2]):
                intersection = 0,0
            else:
                intersection = max(row_objects[i]["bbox"][0],row_objects[i+1]["bbox"][0]), min(row_objects[i]["bbox"][2],row_objects[i+1]["bbox"][2])

            x_width = row_objects[i]["bbox"][2] - row_objects[i]["bbox"][0]
            y_width = row_objects[i+1]["bbox"][2] - row_objects[i+1]["bbox"][0]
            intersection_width = intersection[1] - intersection[0]

            if not (intersection_width/x_width and intersection_width/y_width) > col_intersection_limit:
        
                col_ymin = 0 if row_objects[i]["row"] - 1 < 0 else rows[row_objects[i]["row"] - 1]
                col_ymax = rows[row_objects[i]["row"]] if row_objects[i]["row"] != len(rows) else img.size[1]
                
                column = ((row_objects[i]["bbox"][2]  + row_objects[i+1]["bbox"][0])/2, col_ymin, (row_objects[i]["bbox"][2]  + row_objects[i+1]["bbox"][0])/2, col_ymax)
                # draw.line(column, fill="#FF5C5C", width=10)
                cols_by_rows[j].append(column[0])
            

    
    return cols_by_rows
            

In [ ]:
def find_value_range(num, lst):

    if len(lst) == 0:
        return 0
    if num < lst[0]:
        return 0
    for i in range(len(lst) - 1):
        if lst[i] <= num <= lst[i+1]:
            return i+1  # Add 1 to the index since num is between lst[i] and lst[i+1]
    if num > lst[-1]:
        return len(lst)

In [ ]:
def find_span(lines, bbox_length,belonging_threshold):
    indexes = []

    lines = [int(row) for row in lines]
    bbox_length = [int(y) for y in bbox_length]

    if(len(lines) == 0):return indexes

    for i in range(len(lines)+1):

        if i == 0:
            intersection_values = set(range(0, lines[0])).intersection(set(range(bbox_length[0],bbox_length[1])))
            if intersection_values and (max(intersection_values) - min(intersection_values))/(lines[0]) > belonging_threshold:
                indexes.append(i)
        elif i < len(lines):
            intersection_values = set(range(lines[i-1], lines[i])).intersection(set(range(bbox_length[0],bbox_length[1])))
            if intersection_values and (max(intersection_values) - min(intersection_values))/(lines[i] - lines[i-1]) > belonging_threshold:
                indexes.append(i)
        elif i == len(lines):
            intersection_values = set(range(lines[-1], lines[-1] + (bbox_length[1] - bbox_length[0]))).intersection(set(range(bbox_length[0],bbox_length[1])))
            if intersection_values and (max(intersection_values) - min(intersection_values))/(lines[-1] + (bbox_length[1] - bbox_length[0]) - lines[-1]) > belonging_threshold:
                indexes.append(i)

    return indexes


In [ ]:
def group_objects_by_span(objects, span_type):
    groups = []
    for obj in objects:
        span = obj[span_type]
        matched_group = None
        for group in groups:
            if any(r in group[span_type] for r in span) or any(r in span for r in group[span_type]):
                matched_group = group
                break
        if matched_group:
            matched_group['objects'].append(obj)
            matched_group[span_type] = sorted(set(matched_group[span_type] + span))
        else:
            groups.append({
                span_type: span,
                'objects': [obj]
            })
    return groups


In [ ]:
def get_occupied_pixels(bbox):
    x1, y1, x2, y2 = bbox
    occupied_pixels = set()

    for x in range(int(x1), int(x2)):
        for y in range(int(y1), int(y2)):
            occupied_pixels.add((x, y))

    return occupied_pixels

def are_bboxes_colliding(bbox1, bbox2):
    occupied_pixels1 = get_occupied_pixels(bbox1)
    occupied_pixels2 = get_occupied_pixels(bbox2)

    intersection_pixels = occupied_pixels1.intersection(occupied_pixels2)
    collision_ratio = len(intersection_pixels) / min(len(occupied_pixels1), len(occupied_pixels2))

    return collision_ratio > 0.3

def check_collisions(object_list):
    num_objects = len(object_list)

    for i in range(num_objects - 1):
        bbox1 = object_list[i]['bbox']

        for j in range(i + 1, num_objects):
            bbox2 = object_list[j]['bbox']

            if are_bboxes_colliding(bbox1, bbox2):
                return True  # Collisions detected

    return False  # No collisions detected


In [ ]:
def show(image, objects, row_intersection_limit = 0.3, col_intersection_limit = 0.2):
    
    old_objects = objects
    groups = dict()
    img = image.copy()
    draw = ImageDraw.Draw(img) 

    rows = draw_rows(draw, objects, row_intersection_limit) 

    for i in range(len(objects)):
        objects[i]["row_span"] = find_span(rows,[objects[i]["bbox"][1],objects[i]["bbox"][3]],0.5)
        if len(objects[i]["row_span"]) == 0:
            objects[i]["row_span"] = [find_value_range((objects[i]["bbox"][1] + objects[i]["bbox"][3])/2, rows)]
    
    row_groups = group_objects_by_span(objects,"row_span")

    for i, group in enumerate(row_groups):
        for obj in group["objects"]:
            obj["row"] = i
            obj["pos"].append(i)

    columns = draw_columns(draw, rows, objects, col_intersection_limit)

    for i in range(len(objects)):
        objects[i]["col_span"] = find_span(columns[objects[i]["row"]],[objects[i]["bbox"][0],objects[i]["bbox"][2]],0.2)
        if len(objects[i]["col_span"]) == 0:
            objects[i]["col_span"] = [find_value_range((objects[i]["bbox"][0] + objects[i]["bbox"][2])/2, columns[objects[i]["row"]])]

    

    for j in range(len(rows)+1):

        row_objects = [objects[i] for i in range(len(objects)) if objects[i]["row"] == j] 

        col_groups = group_objects_by_span(row_objects,"col_span")

        groups[j] = col_groups

        for i, group in enumerate(col_groups):
            for obj in group["objects"]:
                obj["column"] = i
                obj["pos"].append(i)
            

    for row in groups.values():

        for col in row:

            pos_values = set()  # Set to store unique "pos" values
            has_duplicates = False

            for obj in col["objects"]:
                pos = tuple(obj['pos'])  # Convert list to tuple for hashability
                if pos in pos_values:
                    has_duplicates = True
                    break
                pos_values.add(pos)

            if has_duplicates:

                print(f"There are objects with the same 'pos' values at {pos}")
                if not col["objects"] == old_objects:#not check_collisions(col["objects"]) and not col["objects"] == old_objects:
                    print(f"Reiterating on {pos}")
                    show(image, col["objects"], 0.7, 0.2)


    for i in range(len(objects)):

        # draw.text((objects[i]["bbox"][0], objects[i]["bbox"][1] - 15), f"{str(objects[i]['id'])} - {str(objects[i]['label'])} - r{str(objects[i]['row'])} - c{str(objects[i]['column'])}  {'i_r' + str(objects[i]['i_r']) if objects[i]['i_r'] !=None else ''}", fill="red", stroke_width=10)
        draw.text((objects[i]["bbox"][0], objects[i]["bbox"][1] - 15), f"{str(objects[i]['id'])} - {str(objects[i]['label'])} - {'pos' + str(objects[i]['pos'])}", fill="red", stroke_width=5,)
        draw.polygon([(objects[i]["bbox"][0],objects[i]["bbox"][1]), (objects[i]["bbox"][2],objects[i]["bbox"][1]), (objects[i]["bbox"][2],objects[i]["bbox"][3]) , (objects[i]["bbox"][0],objects[i]["bbox"][3])],  outline="#E8F5FF", width=5) 
    

    return img


In [ ]:
img_grid = show(img, objects)
img_grid